# DNN Practical: Ag Detection by Muon Spectroscopy

In this notebook, we attempt to solve a real problem in physics using a fully connected DNN.

We have a set of spectra from Muon spectroscopy experiments, from which we would like to detect whether or not a certain element is present in a sample. In this notebook, we are going to train a neural network to detect the presence of Ag. Through this practice, we will encounter and overcome two pitfalls in deep learning, **class imbalance** and **local minima**.

In [ ]:
# tensorflow
import tensorflow as tf
from tensorflow import keras
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Dense, Flatten, Dropout

# check version
print('Using TensorFlow v%s' % tf.__version__)
acc_str = 'accuracy' if tf.__version__[:2] == '2.' else 'acc'

# helpers
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
plt.style.use('ggplot')

# need some certainty in data processing
np.random.seed(0)

## Google Cloud Storage Boilerplate

The following two cells have some boilerplate to mount the Google Cloud Storage bucket containing the data used for this notebook to your Google Colab file system. To access the data, you need to:

1. Run the first cell;
2. Follow the link when prompted (you may be asked to log in with your Google account);
3. Copy the Google SDK token back into the prompt and press `Enter`;
4. Run the second cell and wait until the data folder appears.

If everything works correctly, a new folder called `sciml-workshop-data` should appear in the file browser on the left. Depending on the network speed, this may take one or two minutes. Ignore the warning "You do not appear to have access to project ...". If you are running the notebook locally or you have already connected to the bucket, these cells will take no effect.

In [ ]:
# variables passed to bash; do not change
project_id = 'sciml-workshop'
bucket_name = 'sciml-workshop'
colab_data_path = '/content/sciml-workshop-data'

try:
    from google.colab import auth
    auth.authenticate_user()
    google_colab_env = 'true'
    data_path = colab_data_path
except:
    google_colab_env = 'false'
    ###################################################
    ######## specify your local data path here ########
    ###################################################
    data_path = './sciml-workshop-data'

In [ ]:
%%bash -s {google_colab_env} {colab_data_path} {project_id} {bucket_name}

# running locally
if ! $1; then
    echo "Running notebook locally."
    exit
fi

# already mounted
if [ -d $2 ]; then
    echo "Data already mounted."
    exit
fi

# mount the bucket
echo "deb http://packages.cloud.google.com/apt gcsfuse-bionic main" > /etc/apt/sources.list.d/gcsfuse.list
curl https://packages.cloud.google.com/apt/doc/apt-key.gpg | apt-key add -
apt -qq update
apt -qq install gcsfuse
gcloud config set project $3
mkdir $2
gcsfuse --implicit-dirs --limit-bytes-per-sec -1 --limit-ops-per-sec -1 $4 $2

---

# The dataset

### Read raw data

The raw data, which include the constituent elements and the Muon spectra of the samples, are stored in the pickle file `muon/Ag_muon_data.pkl`. We load this file into a `pandas` dataframe and take a quick look.

In [ ]:
# read data
df = pd.read_pickle(data_path + '/muon/Ag_muon_data.pkl')

# print dimensions
print('Number of samples in the dataset: %d' % len(df['Spectra']))
print('Length of spectra for each sample: %d' % len(df['Spectra'][0]))

# print the first few data
df.head(n=5)

In the above table, the `Elements` and the `Spectra` columns show respectively the elements and the spectra of the samples. There are 224,000 samples in the dataset, and each spectrum is a series of 300 positive reals. 

To get a feel for the complexity of picking out signals with Ag in multinary samples, we can plot some random spectra for three representative cases: 

* no Ag
* pure Ag
* Ag-Si binary

Note that we are plotting only the first half of each spectrum. Change `[0:150]` to `[:]` to show the full spectra.

In [ ]:
# conditions to select data
conditions = [
# no Ag
('no Ag', np.where(['Ag' not in elements for elements in df['Elements']])[0]),
# pure Ag
('pure Ag', np.where([['Ag'] == elements for elements in df['Elements']])[0]),
# Ag-Si
('Ag-Si binary', np.where([['Ag', 'Si'] == elements for elements in df['Elements']])[0])
]

# plot
ncond = len(conditions)
nplot = 5 # number of plots per condition
fig, axs = plt.subplots(nplot, ncond, dpi=200, figsize=(ncond * 5, nplot * 2), sharex=True, sharey=True)
plt.subplots_adjust(wspace=.1, hspace=.2)
for icond, cond in enumerate(conditions):
    for iplot, idata in enumerate(np.random.choice(cond[1], nplot)):
        axs[iplot, icond].plot(df['Spectra'][idata][0:150], c='C%d' % icond)
        axs[iplot, icond].set_xlabel('Sample %d: %s' % (idata, cond[0]), c='C%d' % icond)
        axs[iplot, icond].set_ylim(0, 700)

### Extract training data

The input data for our network will be the `Spectra` column, and we can use the `to_list()` method to convert it to a numpy array. A quick inspection over the spectra shows that the second half of each spectrum has mostly vanished and is thus uninformative; therefore, we will *only use the first half of each spectrum for training*. Such simple data preprocessing can help to reduce model scale and improve both accuracy and performance. 

The output data for our network will be a binary-valued one-hot vector: 0 for no Ag in the sample and 1 otherwise. One-hot encoding can be achieved by a simple for-loop.

In [ ]:
###### input ######
# convert the 'Spectra' column to numpy
train_x = np.array(df['Spectra'].to_list())
# only take the first half for training
train_x = train_x[:, 0:150]

###### output ######
# one-hot encoding: whether Ag is in 'Elements'
train_y = np.array(['Ag' in elements for elements in df['Elements']]).astype(int)

# print data shapes
print("Shape of input: %s" % str(train_x.shape))
print("Shape of output: %s" % str(train_y.shape))

---

# Ag-detection by DNN

## 1. Try out a network


### Build and compile

Based on what we have learnt in [03_DNN_basics.ipynb](03_DNN_basics.ipynb), design a simple neural network with `Dense` layers to detect Ag in the spectra. In general, it is not a straightforward task to determine the number of hidden layers and the number of neurons in each layer, which usually involves some trial and error. In this case, our output size is 1, so we'd better add a small layer before it, such as one with size 8; then, between this hidden layer and the input layer (with size 150), we may add another hidden layer with size 64, approximately halfway between 8 and 150. 

Next, compile the model. We can keep using `adam` for the `optimizer` and `['accuracy']` for the `metrics`. For the `loss`, since we are fitting to a range between 0 and 1, we can choose `binary_crossentropy`.


**Suggested Answer** 

<details> <summary>Show / Hide</summary> 
<p>
    
```python
# define the model
model = Sequential()
model.add(Dense(64, input_dim=150, activation='relu'))
model.add(Dense(8, activation='relu'))
model.add(Dense(1, activation='sigmoid'))

# print model summary
model.summary()

# compile the model
model.compile(loss='binary_crossentropy', optimizer='adam', metrics=['accuracy'])
```
    
</p>
</details>


In [ ]:
# define the model
model = Sequential()
model.add(Dense(64, input_dim=150, activation='relu'))
model.add(Dense(8, activation='relu'))
model.add(Dense(1, activation='sigmoid'))

# print model summary
model.summary()

# compile the model
model.compile(loss='binary_crossentropy', optimizer='adam', metrics=['accuracy'])

### Train the model

Since we have not separated a subset of data for validation, we can pass `validation_split=0.2` to `model.fit()`, which then will use the *final* 20% of the dataset for validation. Let us do 20 epochs first.


**Suggested Answer** 

<details> <summary>Show / Hide</summary> 
<p>
    
```python
# train the model
training_history = model.fit(train_x, train_y, epochs=20, batch_size=64, 
                             validation_split=0.2)
```
    
</p>
</details>

In [ ]:
# train the model
training_history = model.fit(train_x, train_y, epochs=20, batch_size=64, 
                             validation_split=0.2)

### Plot training history

For convenience, we define a function to plot a training history:

In [ ]:
# a function to plot training history
def plot_history(training_history):
    # plot accuracy
    plt.figure(dpi=100, figsize=(12, 4))
    plt.subplot(1, 2, 1)
    plt.plot(training_history.history[acc_str], label='Accuracy on training data')
    plt.plot(training_history.history['val_' + acc_str], label='Accuracy on test data')
    plt.legend()
    plt.title("Accuracy")

    # plot loss
    plt.subplot(1, 2, 2)
    plt.plot(training_history.history['loss'], label='Loss on training data')
    plt.plot(training_history.history['val_loss'], label='Loss on test data')
    plt.legend()
    plt.title("Loss")
    plt.show()

Now, plot the training history of the current model. They will look bizarre at this stage, as explained in the forthcoming section.

**Suggested Answer** 

<details> <summary>Show / Hide</summary> 
<p>
    
```python
# plot training history
plot_history(training_history)
```
    
</p>
</details>

In [ ]:
# plot training history
plot_history(training_history)

## 2. Class imbalance

In the above history plot, notice how the accuracy of the model quickly converges to a certain value and then stops improving completely. This value can be well above 50% (above 95% by the suggested answer), implying that the model is actually picking up something from the dataset. More strangely, the accuracy for the validation data can even be higher than that for the training data (always 100% by the suggested answer). These odd results indicate that something could be wrong within our dataset.

### Data distribution

Let us inspect the distribution of the data using `plt.hist(train_y)`, paying special attention to the validation part (the final 20%).

In [ ]:
# plot distribution of data
plt.figure(dpi=100)
plt.hist(train_y, label='Whole dataset')
plt.hist(train_y[-len(train_y)//5:], label='Validation subset')
plt.xticks([0, 1], ['0: no Ag', '1: with Ag'])
plt.xlabel('label')
plt.ylabel('number of data')
plt.legend()
plt.show()

The histograms show that our dataset is dominated by samples labelled 0 or "no Ag", which account for over 95% of the data. Thus, if the model simply learns to *guess* "no Ag" in every sample, it can achieve 95% accuracy (and 100% for the validation subset) without learning anything meaningful. This problem is known as **class imbalance**.

To avoid this, we must balance the classes. There are a number of strategies we can take:

* Upsample the minority class;
* Downsample the majority class;
* Change the performance metric.

The best available option for our problem is to downsample the majority class, which can be easily achieved with `numpy`:

In [ ]:
# find original indices of 0 ('no Ag') and 1 ('with Ag')
id_no_Ag = np.where(train_y == 0)[0]
id_with_Ag = np.where(train_y == 1)[0]

# downsample 'no Ag' to the number of 'with Ag' by np.random.choice
id_no_Ag_downsample = np.random.choice(id_no_Ag, len(id_with_Ag))

# concatenate 'with Ag' and downsampled 'no Ag'
id_downsample = np.concatenate((id_with_Ag, id_no_Ag_downsample))

# shuffle the indices because they are ordered after concatenation
np.random.shuffle(id_downsample)

# finally get the balanced data
train_x_balanced = train_x[id_downsample]
train_y_balanced = train_y[id_downsample]

Re-exam the histograms of the balanced dataset after downsampling the majority:

In [ ]:
# plot distribution of downsampled data
plt.figure(dpi=100)
plt.hist(train_y_balanced, label='Whole dataset')
plt.hist(train_y_balanced[-len(train_y_balanced)//5:], label='Validation subset')
plt.xticks([0, 1], ['0: no Ag', '1: with Ag'])
plt.xlabel('label')
plt.ylabel('number of data')
plt.legend()
plt.show()

### Re-train the model

Now we can re-train the model with the balanced dataset. Simply change `train_x` and `train_y` to `train_x_balanced` and `train_y_balanced` in `model.fit()` and repeat all the steps in [1. Try out a network](#1.-Try-out-a-network). Note that, to avoid the influence of the initial model state (weights and biases) left by the previous training (such as the one trained with the imbalanced dataset), we have to first re-define and re-compile the model before calling `model.fit()`. A larger `epochs` can be used because we now have much fewer data.

In the following suggested answer, we will first specify the **random seed** before building up the network. This will initialise the model parameters to reproducible random numbers so that we can address another pitfall in the next section: local minima.

**Suggested Answer** 

<details> <summary>Show / Hide</summary> 
<p>
    
```python
# set a random seed
# 0: lead to success; model converges to the global minimum
# 1: lead to failure; model converges to a local minimum
tf.random.set_seed(0)

# re-define the model
model = Sequential()
model.add(Dense(64, input_dim=150, activation='relu'))
model.add(Dense(8, activation='relu'))
model.add(Dense(1, activation='sigmoid'))

# re-compile the model
model.compile(loss='binary_crossentropy', optimizer='adam', metrics=['accuracy'])

# re-train the model
training_history_balanced = model.fit(train_x_balanced, train_y_balanced, 
                                      epochs=200, batch_size=64, validation_split=0.2)

# plot training history
plot_history(training_history_balanced)
```
    
</p>
</details>

In [ ]:
# set a random seed
# 0: lead to success; model converges to the global minimum
# 1: lead to failure; model converges to a local minimum
tf.random.set_seed(0)

# re-define the model
model = Sequential()
model.add(Dense(64, input_dim=150, activation='relu'))
model.add(Dense(8, activation='relu'))
model.add(Dense(1, activation='sigmoid'))

# re-compile the model
model.compile(loss='binary_crossentropy', optimizer='adam', metrics=['accuracy'])

# re-train the model
training_history_balanced = model.fit(train_x_balanced, train_y_balanced, 
                                      epochs=200, batch_size=64, validation_split=0.2)

In [ ]:
# plot training history
plot_history(training_history_balanced)

## 3. Local minima

Let us re-run the above cell using the suggested answer with the random seed set to 1 (also with fewer epochs):

In [ ]:
# set a random seed
# 0: lead to success; model converges to the global minimum
# 1: lead to failure; model converges to a local minimum
tf.random.set_seed(1)

# re-define the model
model = Sequential()
model.add(Dense(64, input_dim=150, activation='relu'))
model.add(Dense(8, activation='relu'))
model.add(Dense(1, activation='sigmoid'))

# re-compile the model
model.compile(loss='binary_crossentropy', optimizer='adam', metrics=['accuracy'])

# re-train the model
training_history_balanced = model.fit(train_x_balanced, train_y_balanced, 
                                      epochs=50, batch_size=64, validation_split=0.2)

In [ ]:
# plot training history
plot_history(training_history_balanced)

This time, the accuracy never improves but only oscillates around 0.5. Let us see what happens if we use this problematic model to predict:

In [ ]:
# use the first 20 data for prediction
print(model.predict(train_x_balanced[0:20]))

We can see that the predicted values are close to neither 0 nor 1, but a constant number near 0.5, so the model has learnt nothing. Since we are using `sigmoid` as the activation function for the output layer, the pre-activation values of the output layer must be a constant number near 0. Therefore, we can infer that the weights and the biases are *all converging to 0* as the model trains, which is a **local minimum** of this problem. 

To avoid this local minimum, we can start from an initial state away from it.

### Model initialisation

By default, Keras initialises the weights in a `Dense` layer by random numbers sampled from the Glorot uniform distribution and the biases by zeros. Such an initial state is stochastically equivalent to the "all-zero" local minimum and thus can easily be dragged into it. 

We can avoid this local minimum by initialising the weights and the biases to be *stochastically* away from "all-zero". This can be done by passing `kernel_initializer` and `bias_initializer` to the constructor of `Dense`. For example, here we can use a normal distribution with a non-zero mean for the weights and a non-zero constant for the biases. Because such an initial state may also get further away the desired global minimum, compared to the default initial state, we will need more epochs for the model to converge.

**Suggested Answer** 

<details> <summary>Show / Hide</summary> 
<p>
    
```python
######## define the initializers ########
# use a normal distribution with a non-zero mean for the weights
init_weights = keras.initializers.RandomNormal(mean=0.1, stddev=0.05)
# use a non-zero constant for the biases
init_biases = keras.initializers.Ones()

# re-define the model with the initializers
model = Sequential()
model.add(Dense(64, input_dim=150, activation='relu',
                kernel_initializer=init_weights, bias_initializer=init_biases))
model.add(Dense(8, activation='relu',
                kernel_initializer=init_weights, bias_initializer=init_biases))
model.add(Dense(1, activation='sigmoid'))

# re-compile the model
model.compile(loss='binary_crossentropy', optimizer='adam', metrics=['accuracy'])

# re-train the model
training_history_balanced = model.fit(train_x_balanced, train_y_balanced, 
                                      epochs=500, batch_size=64, validation_split=0.2)

# plot training history
plot_history(training_history_balanced)
```
    
</p>
</details>

In [ ]:
######## define the initializers ########
# use a normal distribution with a non-zero mean for the weights
init_weights = keras.initializers.RandomNormal(mean=0.1, stddev=0.05)
# use a non-zero constant for the biases
init_biases = keras.initializers.Ones()

# re-define the model with the initializers
model = Sequential()
model.add(Dense(64, input_dim=150, activation='relu',
                kernel_initializer=init_weights, bias_initializer=init_biases))
model.add(Dense(8, activation='relu',
                kernel_initializer=init_weights, bias_initializer=init_biases))
model.add(Dense(1, activation='sigmoid'))

# re-compile the model
model.compile(loss='binary_crossentropy', optimizer='adam', metrics=['accuracy'])

# re-train the model
training_history_balanced = model.fit(train_x_balanced, train_y_balanced, 
                                      epochs=500, batch_size=64, validation_split=0.2)

In [ ]:
# plot training history
plot_history(training_history_balanced)

The history shows that our model is sightly overfitting, with the accuracy for the training data converging to 0.66 and that for the test data to 0.64. This can be alleviated by a dropout layer with a rate of 0.1 after the first hidden layer: try adding `model.add(Dropout(.1))` after `model.add(Dense(64, ...))`. Note that a larger dropout rate may again lead to the local minimum. 

---

## Exercises

Build a DNN to detect the presence of all the elements. To do this, you may go through the following steps:

1. Find all the elements appearing in the dataset; the answer will be `['Zn', 'Sb', 'Si', 'Fe', 'Ag', 'Cu', 'Bi']`.
2. Balance the dataset: if one of the elements appears much less times than the others, it is better to ignore it. Doing everything correctly, you will find the number of samples containing each element as shown in the following table. Therefore, we may ignore Ag in this network.


|  Element | # Samples |
|---|---|
|Zn| 91935|  
|Sb| 91880|  
|Si| 91672| 
|Fe| 91446|
|Ag| 8617|
|Cu| 92133|
|Bi| 91796|
    
3. Do one-hot encoding for the element list `['Zn', 'Sb', 'Si', 'Fe', 'Cu', 'Bi']`; if a sample contains Fe and Sb, e.g., the one-hot vector for this sample will be `[0, 1, 0, 1, 0, 0]`.
4. Build and train a DNN (with an input size of 150 and an output size of 6) to detect the presence of the six elements.

Do not feel disappointed if the accuracy of the DNN turns out low, as some of the elements are physically indistinguishable from the given spectra. Nevertheless, our DNN can still mine useful information from the dataset. By comparing the predictions and the ground truth, you may notice that **only one element** is dominating the training, that is, the network can correctly predict the presence of this element while the predictions for the other elements are basically random. This indicates that the given set of spectra is characterised by this element (not considering Ag). Figure out what the element is. 
